# DATA 271 Final Project: US Homelessness

Homelessness in America, and specifically in California, is a major issue that affects hundreds of thousands of people. I would like to look into the data involving homelessness and see how many people are affected. I think that, through looking at trends in this data, I can try to look at real life situations and causes that might be affecting these trends. Such examples could include policies and legislations, and other potential socioeconomic factors.
If I can see potentially correlating factors in homeless population trends, I may be able to link them to outside circumstances and suggest potential changes.

This data informs policymakers as to how to delegate help to homeless populations, so it is important to understand it to get the proper funding.

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install openpyxl
!pip install pyxlsb

-  **numpy**: For any calculations I may need to do.
-  **pandas**: For importing data and utilizing it in a dataframe.
-  **matplotlob**: For visualizing data.
-  **seaborn**: For visualizing data.
__________________________________________________________


-  **pit_counts_df** (https://www.huduser.gov/portal/sites/default/files/xls/2007-2024-PIT-Counts-by-State.xlsb): This is a dataset of Point-In-Time (PIT) estimates by states in the United States, from 2007 to 2024. Point-In-Time count is an effort led by the U.S. Department of Housing and Urban Development, also known as HUD, that estimates the number of homeless people in the United States. I will be looking at the years 2014-2024.
-  This dataset has hundreds, and sometimes over a thousand variables, so I will need to narrow them down to the most relevant ones. I'll be looking at ages of homeless people, amounts of homeless people, CoC numbers, etc.
_____________________________

-  **hic_counts_df** (https://www.huduser.gov/portal/sites/default/files/xls/2007-2024-HIC-Counts-by-State.xlsx): This is a dataset of Housing Inventory Counts (HIC) by states in the United States, from 2007 to 2024. HIC is an inventory count system used within a CoC that provides beds dedicated to serving homeless people. I will be looking at the years 2014-2024.
-  This dataset has many variables dedicated to bed counts, unit counts, specific categories for children, adults, veterans, etc.
- Some categories of beds include **ES (emergency shelter), TH (transitional housing), SH (safe haven), and OPH (other permanent housing).**
__________________________________________________________


In [31]:
pit_counts_df = pd.read_excel('2007-2024-PIT-Counts-by-State.xlsb', engine='pyxlsb', sheet_name=None)

hic_counts_df = pd.read_excel('2007-2024-HIC-Counts-by-State.xlsx', engine='openpyxl', sheet_name=None,header=None)

/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""C

In [34]:
base_pit_copy = pit_counts_df.copy()
base_hic_copy = hic_counts_df.copy()

**hic_counts_df** is having some problems with the headers, so I'll clean them up.

In [35]:
with pd.ExcelFile('2007-2024-HIC-Counts-by-State.xlsx') as excel:
    hic_dict = pd.read_excel(excel, sheet_name=None, header=None)

processed_sheets = {}
for sheet_name, df in hic_dict.items():
    header_row = 1 
    if len(df) > header_row: 
        df.columns = df.iloc[header_row].values
        df = df.drop([0, 1]).reset_index(drop=True)
        processed_sheets[sheet_name] = df.drop(header_row).reset_index(drop=True)
    else:
        processed_sheets[sheet_name] = df 

with pd.ExcelWriter('2007-2024-HIC-Counts-by-State-modified.xlsx') as writer:
    for sheet_name, df in processed_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""C

In [62]:
# creating a new .xlsx file with the edited headers
hic_counts_df = pd.read_excel('2007-2024-HIC-Counts-by-State-modified.xlsx', engine='openpyxl', sheet_name=None)

_______________________________________________________________________________

**hic_counts_df** now properly has the correct header in each sheet.
_______________________________________________________________________________

In [63]:
# reducing the pit_counts_df dataset to the years 2014-2024

pit_decade = list(pit_counts_df.items())[:11]

updated_dfs_pit = []

for sheet_name, df in pit_decade:
    year = int(sheet_name)
    df["Year"] = year
    updated_dfs_pit.append(df) 

# combining each dataset now that each row denotes the year
pit_concat_df = pd.concat(updated_dfs_pit, ignore_index=True) 

In [72]:
# reducing the hic_counts_df dataset to the years 2014-2024

hic_decade = list(hic_counts_df.items())[:11]

updated_dfs_hic = []

for sheet_name, df in hic_decade:
    year = int(sheet_name)
    df["Year"] = year
    updated_dfs_hic.append(df) 

# combining each dataset now that each row denotes the year
hic_concat_df = pd.concat(updated_dfs_hic, ignore_index=True) 